
Create a Minecraft Server on Google Colab!

The script below will initiate your server. First, you will need to create the server to be able to use it - don't worry, the scripts below will do the majority of the work for you. Perhaps you want to change the server's region, check below.

Original: https://github.com/devrock07/minecolab
Modified Version Well Optimized

```
███████╗███████╗███╗   ██╗██╗████████╗██╗  ██╗███████╗███████╗███╗   ██╗██████╗  █████╗ ██╗
╚══███╔╝██╔════╝████╗  ██║██║╚══██╔══╝██║  ██║██╔════╝██╔════╝████╗  ██║██╔══██╗██╔══██╗██║
  ███╔╝ █████╗  ██╔██╗ ██║██║   ██║   ███████║███████╗█████╗  ██╔██╗ ██║██████╔╝███████║██║
 ███╔╝  ██╔══╝  ██║╚██╗██║██║   ██║   ██╔══██║╚════██║██╔══╝  ██║╚██╗██║██╔═══╝ ██╔══██║██║
███████╗███████╗██║ ╚████║██║   ██║   ██║  ██║███████║███████╗██║ ╚████║██║     ██║  ██║██║
╚══════╝╚══════╝╚═╝  ╚═══╝╚═╝   ╚═╝   ╚═╝  ╚═╝╚══════╝╚══════╝╚═╝  ╚═══╝╚═╝     ╚═╝  ╚═╝╚═╝                                                                                                                        

```


# **1º Create the Server (Only do this the first time)**

The code below will create your server and accept the EULA. When you use these scripts, your server will be ready to start.


**Download the Minecraft Server**

The code below will download Paper, a type of high-performance Vanilla server that allows the use of plugins. If you want to use another type of server.jar, put it in the Drive folder manually.

The server files will be saved on your Google Drive (in the Google account you choose or are using).

In [ ]:
# Change "1.21" to the desired version.
# Available tested versions:
# - 1.21
# - 1.20.2
# - 1.20.1
# Newer versions might work too, however this isn't guaranteed.

version = '1.21'

# Choose server type. Currently available versions:
# - paper (Most vanilla. It's the same as SPIGOT, bukkit, etc.)
# - forge (For using MODS. It's basically FORGE) [Note, installation takes approximately 10-15 minutes, wait for "COMPLETED" after executing the first step.]
# - fabric (For playing Fabric. It's an alternative to Forge)

server_type = 'paper'

from google.colab import drive
import requests
import json

drive.mount('/content/drive')

# Create the directory which will be used for the server
! mkdir "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"

# Internal init...

a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
paperURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

forgeURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar', 'forge': 'forge-installer.jar'}
url = {
    'paper': paperURL,
    'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar',
    'forge': forgeURL
}

print('Downloading to Google Drive...')

r = requests.get(url[server_type])

if r.status_code == 200:
    with open('/content/drive/My Drive/Minecraft-server/' + jar_name[server_type], 'wb') as f:
        f.write(r.content)
else:
    print('Error '+ str(r.status_code) + '! Most likely you entered an unsupported version. Try running the code again if you think that shouldn\'t have happened.')

# Running specific install path.
if server_type == 'fabric':
    !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

if server_type == 'forge':
    %cd "/content/drive/My Drive/Minecraft-server"
    !java -jar forge-installer.jar --installServer

# Saving config
colabconfig = {"server_type": server_type,
               "server_version": version}
json.dump(colabconfig, open("colabconfig.json",'w'))

print('Completed!')  # todo: Would show even after erroring.

# Please read the file stored in your server folder before running this command.
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt


# **-> Start the Minecraft server - Use this whenever you want to open it.**



In [ ]:
import os
import json
from google.colab import drive

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

# Mount Google Drive
drive.mount('/content/drive')

# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls # List the directory contents (to verify that working directory was changed)

# Import config file
if os.path.isfile("colabconfig.json"):
    colabconfig = json.load(open("colabconfig.json"))
else:
    colabconfig = {"server_type": "generic"} # using default, if config doesn't exist.
    json.dump(colabconfig, open("colabconfig.json",'w'))

# Install OpenJDK 21
!sudo apt-get install openjdk-21-jre-headless &>/dev/null && echo "Yay! Openjdk21 has been successfully installed." || echo "Failed to install OpenJdk21."

# Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "21":
    print("Openjdk21 is working correctly, you are good to go.")
else:
    print("Openjdk21 doesn't seem to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance. Minecraft 1.21 and above might fail to launch.")

# Server jar names
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments
if colabconfig["server_type"] == "paper":
    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
    server_flags = "" # Aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xms8704M -Xmx8704M"

# Choose the tunnel service you want to use
# Available options: ngrok, argo, playit
tunnel_service = "playit"
print("Using", tunnel_service)

!java $memory_allocation $server_flags -jar $jar_name nogui


**FAQ - THINGS YOU NEED TO KNOW**

you just need to download playit.gg plugin and from there you can make a static ip.

open google drive for adding plugins and other files modification.